In [41]:
import get_mta

%run get_mta "17"


52 weeks to collect:

['2017-12-30', '2017-12-23', '2017-12-16', '2017-12-09', '2017-12-02', '2017-11-25', '2017-11-18', '2017-11-11', '2017-11-04', '2017-10-28', '2017-10-21', '2017-10-14', '2017-10-07', '2017-09-30', '2017-09-23', '2017-09-16', '2017-09-09', '2017-09-02', '2017-08-26', '2017-08-19', '2017-08-12', '2017-08-05', '2017-07-29', '2017-07-22', '2017-07-15', '2017-07-08', '2017-07-01', '2017-06-24', '2017-06-17', '2017-06-10', '2017-06-03', '2017-05-27', '2017-05-20', '2017-05-13', '2017-05-06', '2017-04-29', '2017-04-22', '2017-04-15', '2017-04-08', '2017-04-01', '2017-03-25', '2017-03-18', '2017-03-11', '2017-03-04', '2017-02-25', '2017-02-18', '2017-02-11', '2017-02-04', '2017-01-28', '2017-01-21', '2017-01-14', '2017-01-07']

Continue? (y/n) y

Connecting to (or creating) mta_data SQLite database ...

Week: 2017-12-30 -- Getting data ...
Week: 2017-12-30 -- Inserting into SQLite ...
Week: 2017-12-23 -- Getting data ...
Week: 2017-12-23 -- Inserting into SQLite ...
Week

In [42]:
import os

import sqlite3 as sq
from sqlalchemy import create_engine, inspect, Table, MetaData
from sqlalchemy.orm import sessionmaker

import pandas as pd


In [43]:
mta_data_engine = create_engine("sqlite:///mta_data.db")


In [44]:
def insert_table_from_csv(csv_list, engine):
    for file in csv_list:
        with open(file, 'r') as f:
            data = pd.read_csv(f)
            data.columns = data.columns.str.strip()
        data.to_sql(os.path.splitext(file)[0], con=mta_data_engine, index=False, if_exists='replace')
    

In [45]:
insert_table_from_csv(['mta_stations_loc.csv', 'mta_complex_id.csv'], mta_data_engine)


In [46]:
insp = inspect(mta_data_engine)
print(insp.get_table_names())


['mta_complex_id', 'mta_data', 'mta_stations_loc']


In [49]:
pd.read_sql('''
            ALTER TABLE mta_data 
            RENAME COLUMN "C/A" TO "BOOTH";
            ''',
            mta_data_engine)

,1


In [50]:
temp_df = pd.read_sql('''
            SELECT a.booth, a.unit, a.scp, a.station, a.linename, a.division, 
              MAX(a.entries)-MIN(a.entries) AS net_entries,
              MAX(a.exits)-MIN(a.exits) AS net_exits,
              CAST(b.complex_id AS int) AS complex_id, 
              c."GTFS Latitude" AS lat, c."GTFS Longitude" AS lon
            FROM mta_data a
            JOIN mta_complex_id b
            JOIN mta_stations_loc c
            ON a.booth = b.booth AND a.unit = b.remote AND b.complex_ID = c."Complex ID"
            WHERE a.date >= '06/01/2017' 
              AND a.date <= '08/31/2017' 
              AND a.desc = 'REGULAR'
            GROUP BY a.booth, a.unit, a.scp;
            ''', 
            mta_data_engine)


In [51]:
mta_df = pd.read_sql('''
            SELECT booth, unit, station, linename, division, SUM(net_entries)+SUM(net_exits) AS net_rides, complex_id, lat, lon
            FROM
            (SELECT a.booth, a.unit, a.scp, a.station, a.linename, a.division, 
              MAX(a.entries)-MIN(a.entries) AS net_entries,
              MAX(a.exits)-MIN(a.exits) AS net_exits,
              CAST(b.complex_id AS int) AS complex_id, 
              c."GTFS Latitude" AS lat, c."GTFS Longitude" AS lon
            FROM mta_data a
            JOIN mta_complex_id b
            JOIN mta_stations_loc c
            ON a.booth = b.booth AND a.unit = b.remote AND b.complex_ID = c."Complex ID"
            WHERE a.date >= '06/01/2017' 
              AND a.date <= '08/31/2017' 
              AND a.desc = 'REGULAR'
            GROUP BY a.booth, a.unit, a.scp)
            GROUP BY complex_id;
            ''', 
            mta_data_engine)

mta_df

,booth,unit,station,linename,division,net_rides,complex_id,lat,lon
0,R515,R095,ASTORIA DITMARS,NQW,BMT,2281481,1,40.775036,-73.912034
1,R514,R094,ASTORIA BLVD,NQW,BMT,1551079,2,40.770258,-73.917843
2,R513,R093,30 AV,NQW,BMT,1877469,3,40.766779,-73.921479
3,R512,R092,BROADWAY,NQW,BMT,1632997,4,40.761820,-73.925508
4,R511,R091,36 AV,NQW,BMT,884227,5,40.756804,-73.929575
...,...,...,...,...,...,...,...,...,...
418,N095,R014,FULTON ST,ACJZ2345,IND,59564170,628,40.709416,-74.006571
419,H012,R268,LORIMER ST,LG,BMT,3536718,629,40.712792,-73.951418
420,H026,R137,MYRTLE-WYCKOFF,LM,BMT,3155544,630,40.699430,-73.912385
421,A058,R001,WHITEHALL S-FRY,R1W,BMT,7486428,635,40.702068,-74.013664
